# Decision analysis with Julia `DecisionProgramming.jl` using Evans, 1997  
Tomás Aragón, Updated 2026-01-21

I mostly replicated R `rdecision` vignette "Elementary decision tree (Evans 1997)": https://cran.r-project.org/web/packages/rdecision/vignettes/DT01-Sumatriptan.html . 

Sources:
- Briggs, Andrew Harvey, Karl Claxton, and Mark Sculpher. Decision Modelling for Health Economic Evaluation. Repr. [d. korr. Ausg. von 2007]. Handbooks in Health Economic Evaluation Series. Oxford Univ. Press, 2011.
- Evans, K. W., J. A. Boan, J. L. Evans, and A. Shuaib. “Economic Evaluation of Oral Sumatriptan Compared with Oral Caffeine/Ergotamine for Migraine.” PharmacoEconomics 12, no. 5 (1997): 565–77. https://doi.org/10.2165/00019053-199712050-00007.

This Julia Jupyter notebook was created in VS Code and contains examples from Evans, 1997. You can run this notebook in VS Code, Positron, or JupyterLab with a Julia kernel. To learn about the Julia language visit https://julialang.org/ .

We will use `DecisionProgramming.jl` Julia package: https://gamma-opt.github.io/DecisionProgramming.jl/dev/ . `DecisionProgramming.jl` is a Julia package for modeling and solving decision analysis problems using influence diagrams.

Preparation: 
- PENDING


Optional readings for background:
- Owens, Douglas K. “Analytic Tools for Public Health Decision Making.” Medical Decision Making: An International Journal of the Society for Medical Decision Making 22, no. 5 Suppl (2002): S3-10. https://doi.org/10.1177/027298902237969.
- Owens, D. K., R. D. Shachter, and R. F. Nease. “Representation and Analysis of Medical Decision Problems with Influence Diagrams.” Medical Decision Making: An International Journal of the Society for Medical Decision Making 17, no. 3 (1997): 241–62. https://doi.org/10.1177/0272989X9701700301.
- Nease, R. F., and D. K. Owens. “Use of Influence Diagrams to Structure Medical Decisions.” Medical Decision Making: An International Journal of the Society for Medical Decision Making 17, no. 3 (1997): 263–75. https://doi.org/10.1177/0272989X9701700302.
- Neapolitan, Richard, Xia Jiang, Daniela P. Ladner, and Bruce Kaplan. “A Primer on Bayesian Decision Analysis With an Application to a Kidney Transplant Decision.” Transplantation 100, no. 3 (2016): 489–96. https://doi.org/10.1097/TP.0000000000001145.

## Creating the model
"This vignette is an example of modelling a decision tree using the rdecision package. It is based on the example given by Briggs (Box 2.3) which itself is based on a decision tree which compared oral Sumatriptan versus oral caffeine/Ergotamine for migraine. In this vignette, we consider the problem from the perspective of a provincial health department."

<figure>
<img src="img_Briggs2006_Box2-3.png" width="700" alt="Evans 1997 Figure 1"/>
<figcaption>BOX 2.3. Decision tree from Briggs et al., 2006.</figcaption>
</figure>

### Figure 1 from Evans, 1997
Below is Figure 1 from Evans, 1997 article showing the decision tree structure. 

<figure>
<img src="img_evans1997_fig01.png" width="800" alt="Evans 1997 Figure 1"/>
<figcaption>FIGURE 1: From Evans, 1997, Figure 1.</figcaption>
</figure>

Figure 2 is the influence diagram of Figure 1. 

<figure>
<img src="img_drawio_evans1997_Influence_Diagram.png" width="700" alt="Evans 1997 Figure 1"/>
<figcaption>FIGURE 2. Influence diagram derived from Evans, 1997, Figure 1.</figcaption>
</figure>

### Model variables
The following code defines the variables for cost, utility and effect that will be used in the model. There are 14 variables in total; 4 costs, 4 utilities and 6 probabilities.

For clarity, I coded some of the variables differently than in the original vignette. For example,

`p_norecurrence_relief_sumatriptan = 0.594` $= P(\text{no recurrence} \mid \text{relief}, \text{sumatriptan})$.

For example, 

`p_endures_norelief = 0.92` $= P(\text{endures} \mid \text{no relief})$ applies to both sumatriptan and caffeine/ergotamine treatment arms, so this variable appears in both arms.

## Set nodes, incoming edges, and states

In [1]:
using JuMP, HiGHS
using DecisionProgramming

### Set nodes with name, incoming edges, and states 

For example, consider,

`add_node!(diagram, ChanceNode("E", ["D", "R"], ["ED", "En"]))` 

- `"E"` is the name of the node
- `["D", "R"]` are the incoming edges from node D (Decision/Treatment) and node R (Relief vs No relief). This is called the "information set" of node M.
- `["ED", "En"]` are the states of node E: "ED" = Emeergency Department, "En" = Endure

I chose to use "M1" and "M0" because it is much easier to read in the code.

In [6]:
diagram = InfluenceDiagram()
add_node!(diagram, DecisionNode("D", [], ["S", "C"])) # Decision: Sumatriptan (S) vs Coffeine/Ergot (C)
add_node!(diagram, ChanceNode("R", ["D"], ["R1", "R0"])) # R1=relief, R0=no relief  
add_node!(diagram, ChanceNode("N", ["D", "R"], ["N1", "N0"])) # N1=non-recurrence, N0=recurrence
add_node!(diagram, ChanceNode("E", ["D", "R"], ["E1", "E0"])) # E1=ED, E0=endures
add_node!(diagram, ChanceNode("H", ["D", "E"], ["H1", "H0"])) # H1=hospital, H0=relief 
add_node!(diagram, ValueNode("V", ["N", "E", "H"]));

### Generate arcs 

In [4]:
generate_arcs!(diagram) 

OrderedCollections.OrderedDict{String, Utilities}()

### options for viewing diagram attributes  
`diagram.OPTION` where `OPTION` = `Nodes`, `Names`, `I_j`, `States`, `S`, `C`, `D`, or `V`.

In [5]:
diagram.I_j

OrderedCollections.OrderedDict{String, Vector{String}} with 6 entries:
  "D" => []
  "R" => ["D"]
  "N" => ["D", "R"]
  "E" => ["D", "R"]
  "H" => ["D", "E"]
  "V" => ["N", "E", "H"]

## Assign probabilities to variables
For clarity, here is how the notation works:

REDO BELOW

- `p_E1` $=P(E = 1) = 0.20$
- `p_M1_E1_R1` $= P(M = 1 \mid E = 1, R = 1) = 0.05$
- `p_M1_E1_R0` $= P(M = 1 \mid E = 1, R = 0) = 0.33$
- `p_D1_M1` $= P(D = 1 \mid M = 1) = 0.0023$

In [ ]:

# model variables for cost
c_sumatriptan = 16.10 #
c_caffeine = 1.32 #
c_ed = 63.16 #
c_hospital = 1093.0 #

# model variables for utility
u_relief_norecurrence = 1.0 # 
u_relief_recurrence = 0.9 #
u_norelief_endures = -0.30 #
u_norelief_ed = 0.1 #

# model variables for effect; p_norecurrence_relief_sumatriptan = P(no recurrence | relief, sumatriptan)
p_R1_S = 0.558
p_R0_S = 1 - p_R1_S
p_R1_C = 0.379
p_R0_C = 1 - p_R1_C 
p_N1_R1_S = 0.594 # e07; # e08 use NA_real_ to indicate complement
p_N0_R1_S = 1 - p_N1_R1_S 
p_endures_norelief = 0.92 # e09, e13; # e10, e14 use NA_real_ to indicate complement
p_norecurrence_relief_caffeine = 0.703 # e11; # e12 use NA_real_ to indicate complement
p_relief_ed = .998 # e15, e17; # e16, e18 use NA_real_ to indicate complement